In [ ]:
%load_ext autoreload
%autoreload 2
from pylab import *
import pandas as pd
from mpl_arrow import arrow,arrow_absolute,vector

cmap_name = 'cividis'
from tessellography import UnitCell,Molecule

molecule = Molecule.from_pdb("serotonin.pdb", random_state=1)

color_scheme = {
    'C' : 'w',#"0.5",
    'N' : 'b',
    'O' : 'r',
}

gamma = 75.
padding = 0.2
cell = UnitCell.from_molecule(molecule, gamma, padding)
molecule.plot(color_scheme=color_scheme)
cell.plot_cell('--w')
cell.plot_a(color='k')
cell.plot_b(color='k')
ax = plt.gca()
ax.set_facecolor('k')

In [ ]:
contours=100
dmin = 1.2

#Defines the width of the atoms
b = 3.

#Fractional grids
npoints = 100
fgrid = np.mgrid[
    -1.0:2.0:1./npoints,
    -1.0:2.0:1./npoints,
].reshape((2, -1)).T
real_grid = (cell.O@fgrid.T).T

#Get all hkls
H = cell.get_reciprocal_cell(dmin)

d = cell.get_resolution(H)
scattering_factors = np.exp(-b / (d * d))
coords = np.column_stack((molecule.x, molecule.y))

#Atom coords in fractional
X = cell.F@coords.T

#Structure factor summation
F = np.sum(scattering_factors[...,None]*np.exp(2j*np.pi*(H@X)), axis=-1)

#FT to real space
rho = (F[...,None]*np.exp(-2j*np.pi*H@fgrid.T))

rho_tot = rho.sum(-2)

#Electron density plot
x,y = real_grid.T
z = rho_tot.real
z = (z - z.mean()) / z.std()
plt.tricontourf(x, y, z, contours, cmap=cmap_name)
ax = plt.gca()
ax.set_facecolor(plt.get_cmap(cmap_name)(0.))
molecule.plot()
cell.plot_cell(fmt='--w')
cell.crop_cell()

In [ ]:
from celluloid import Camera
gif_out_file = None
phase_cmap_name='twilight'

f, axd = plt.subplot_mosaic([
        ['left', 'upper right'],
        ['left', 'lower right'],
    ],
    figsize=1.5 * np.array((5., 3.)),
    #layout="constrained",
)
ax1 = axd['left']
ax2 = axd['upper right']
ax3 = axd['lower right']
#f,(ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(10,5))

ax1.set_aspect('equal')
ax2.set_aspect('equal')
ax3.set_aspect('equal')


x,y = cell.B@H.T
size_min,size_max = 0.5, 5.
I = np.square(np.abs(F))
phase = np.angle(F)
size = I - I.min()
size = I / I.max()
size = (size_max - size_min)*I + size_min

ax1.set_xticks([])
ax1.set_yticks([])
ax2.set_xticks([])
ax2.set_yticks([])
ax3.set_xticks([])
ax3.set_yticks([])

ax1.set_title("Diffraction Pattern")
ax2.set_title("Single Reflection Density")
ax3.set_title("Cumulative Electron Density")


vmax = rho.real.max()
vmin = -vmax

camera = Camera(f)
result = None
spots = []
from tqdm import tqdm

phase = np.rad2deg(phase)

from matplotlib.cm import ScalarMappable
sm = ScalarMappable(
    plt.Normalize(-180., 180.),
    plt.get_cmap(phase_cmap_name), 
)
shrink_cbar = 0.5
fraction_cbar = 0.1
cbar = plt.colorbar(sm, ax=ax1, orientation='horizontal', label='Phase ($^\circ$)', shrink=shrink_cbar, fraction=fraction_cbar)
for i in tqdm(np.argsort(1./d)):
    z = rho[i].real
    ax1.scatter(x, y, s=size, c=phase, cmap=phase_cmap_name, vmin=-180., vmax=180.)
    ax1.set_facecolor('k')
    plt.colorbar(sm, cax=cbar.ax, orientation='horizontal', label='Phase ($^\circ$)', shrink=shrink_cbar, fraction=fraction_cbar)
    spots.append(i)
    ax1.scatter(x[spots], y[spots], s=size[spots], facecolor='r')
    ax1.scatter(x[i], y[i], s=1.5*size[i], facecolor='None', edgecolor='w')
    rx,ry = real_grid.T
    ax2.tricontourf(rx, ry, z, contours, vmin=vmin, vmax=vmax, cmap=cmap_name)
    molecule.plot(color_scheme=color_scheme, ax=ax2)
    if result is None:
        result = np.zeros_like(z)
    result = z + result
    zscore = (result - result.mean()) / result.std()
    ax3.tricontourf(rx, ry, zscore, contours, cmap=cmap_name)
    molecule.plot(color_scheme=color_scheme, ax=ax3)
    
    
    ax2.set_facecolor(plt.get_cmap(cmap_name)(0.))
    ax3.set_facecolor(plt.get_cmap(cmap_name)(0.))
    cell.plot_cell(ax=ax2, fmt='--w')
    cell.plot_cell(ax=ax3, fmt='--w')
    cell.crop_cell(ax=ax2)
    cell.crop_cell(ax=ax3)
    plt.tight_layout()
    camera.snap()

plt.close() # Don't show static canvas
anim = camera.animate()
dpi = 300
if gif_out_file is not None:
    anim.save(gif_out_filele, dpi=dpi)

# Embed the movie as HTML
from IPython.display import HTML
anim = camera.animate()
HTML(anim.to_html5_video())

In [ ]:
from celluloid import Camera

phase_cmap_name='twilight'

f, axd = plt.subplot_mosaic([
        ['left', 'right'],
    ],
    figsize=1.5 * np.array((5., 3.)),
    #layout="constrained",
)
ax1 = axd['left']
ax2 = axd['right']
#f,(ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(10,5))

ax1.set_aspect('equal')
ax2.set_aspect('equal')


x,y = cell.B@H.T
size_min,size_max = 0.5, 5.
I = np.square(np.abs(F))
phase = np.angle(F)
size = I - I.min()
size = I / I.max()
size = (size_max - size_min)*I + size_min

ax1.set_xticks([])
ax1.set_yticks([])
ax2.set_xticks([])
ax2.set_yticks([])

ax1.set_title("Diffraction Pattern")
ax2.set_title("Single Reflection Density")


vmax = rho.real.max()
vmin = -vmax

camera = Camera(f)
result = None
spots = []
from tqdm import tqdm

phase = np.rad2deg(phase)

from matplotlib.cm import ScalarMappable
sm = ScalarMappable(
    plt.Normalize(-180., 180.),
    plt.get_cmap(phase_cmap_name), 
)
shrink_cbar = 0.5
fraction_cbar = 0.1
cbar = plt.colorbar(sm, ax=ax1, orientation='horizontal', label=r'Phase ($^\circ$)', shrink=shrink_cbar, fraction=fraction_cbar)

h = [2, 1]
i = np.where(np.all(h == H, axis=1))[0][0]
npoints = 10
phases = np.concatenate((
    np.linspace(-np.pi, np.pi, npoints),
    np.linspace(np.pi, -np.pi, npoints),
))

dinv = cell.B@h
d = dinv / np.square(np.linalg.norm(dinv))
m = d[1] / d[0]
from tqdm import tqdm
for phi in tqdm(phases):
    z = rho[i].real
    
    xlim = ax2.get_xlim()
    root = d * (phi) / 2 / np.pi 
    
    z = rho[i].real
    rx,ry = real_grid.T - d[:,None] * np.angle(F[i]) / 2 / np.pi 
    ax2.tricontourf(rx + root[0], ry + root[1], z, contours, vmin=vmin, vmax=vmax, cmap=cmap_name)
    cell.plot_cell('--w', ax=ax2)
    cell.crop_cell(ax=ax2, padding=.2)
    ax2.plot(
        xlim,
        [m * xlim[0], m * xlim[1]],
        'k-',
        scalex=False,
        scaley=False,
    )
    arrow(root[0], root[1], d[0], d[1], color='w', zorder=10, ax=ax2)
    
    ax2.set_xticks([])
    ax2.set_yticks([])
    
    phase[i] = np.rad2deg(phi)
    ax1.scatter(x, y, s=size, c=phase, cmap=phase_cmap_name, vmin=-180., vmax=180.)
    ax1.set_facecolor('k')
    plt.colorbar(sm, cax=cbar.ax, orientation='horizontal', label=r'Phase ($^\circ$)', shrink=shrink_cbar, fraction=fraction_cbar)
    phase_y = np.mean(cbar.ax.get_ylim())
    cbar.ax.scatter(np.rad2deg(phi), phase_y, color='k', edgecolors='w')
    spots.append(i)
    
    ax1.scatter(x[i], y[i], s=1.5*size[i], facecolor='None', edgecolor='w')
    
    
    dx = x.std() / 3.
    ax1.set_xlim(x[i] - dx, x[i] + dx)
    ax1.set_ylim(y[i] - dx, y[i] + dx)
    
    ax2.set_facecolor(plt.get_cmap(cmap_name)(0.))
    cell.plot_cell(ax=ax2, fmt='--w')
    cell.crop_cell(ax=ax2)
    plt.tight_layout()
    camera.snap()


plt.close() # Don't show static canvas
anim = camera.animate()
dpi = 300
HTML(anim.to_html5_video())

In [ ]:
from celluloid import Camera

phase_cmap_name='twilight'

f, axd = plt.subplot_mosaic([
        ['left', 'right'],
    ],
    figsize=1.5 * np.array((5., 3.)),
    #layout="constrained",
)
ax1 = axd['left']
ax2 = axd['right']
#f,(ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(10,5))

ax1.set_aspect('equal')
ax2.set_aspect('equal')


x,y = cell.B@H.T
size_min,size_max = 0.5, 5.
I = np.square(np.abs(F))
phase = np.angle(F)
size = I - I.min()
size = I / I.max()
size = (size_max - size_min)*I + size_min

ax1.set_xticks([])
ax1.set_yticks([])
ax2.set_xticks([])
ax2.set_yticks([])

ax1.set_title("Diffraction Pattern")
ax2.set_title("Single Reflection Density")



camera = Camera(f)
result = None
spots = []
from tqdm import tqdm

phase = np.rad2deg(phase)

from matplotlib.cm import ScalarMappable
sm = ScalarMappable(
    plt.Normalize(-180., 180.),
    plt.get_cmap(phase_cmap_name), 
)
shrink_cbar = 0.5
fraction_cbar = 0.1
cbar = plt.colorbar(sm, ax=ax1, orientation='horizontal', label=r'Phase ($^\circ$)', shrink=shrink_cbar, fraction=fraction_cbar)

h = [2, 1]
miller_id = np.where(np.all(h == H, axis=1))[0][0]

dinv = cell.B@h
d = dinv / np.square(np.linalg.norm(dinv))
m = d[1] / d[0]
from tqdm import tqdm
phi = phases[i]

#Relative sf amplitude
amin,amax = 0., 3.
npoints=10
amplitudes = np.concatenate((
    np.linspace(amax, amin, npoints),
    np.linspace(amin, amax, npoints),
))

vmax = rho[i].real.max() * amax
vmin = -vmax

for amplitude in tqdm(amplitudes):
    z = rho[miller_id].real
    
    xlim = ax2.get_xlim()
    root = d * (phi) / 2 / np.pi 
    
    z = rho[miller_id].real
    rx,ry = real_grid.T - d[:,None] * np.angle(F[miller_id]) / 2 / np.pi 
    ax2.tricontourf(rx + root[0], ry + root[1], amplitude * z, contours, vmin=vmin, vmax=vmax, cmap=cmap_name)
    cell.plot_cell('--w', ax=ax2)
    cell.crop_cell(ax=ax2, padding=.2)
    ax2.plot(
        xlim,
        [m * xlim[0], m * xlim[1]],
        'k-',
        scalex=False,
        scaley=False,
    )
    arrow(root[0], root[1], d[0], d[1], color='w', zorder=10, ax=ax2)
    
    ax2.set_xticks([])
    ax2.set_yticks([])
    
    ax1.scatter(x, y, s=size, c=phase, cmap=phase_cmap_name, vmin=-180., vmax=180.)
    ax1.set_facecolor('k')
    plt.colorbar(sm, cax=cbar.ax, orientation='horizontal', label=r'Phase ($^\circ$)', shrink=shrink_cbar, fraction=fraction_cbar)
    phase_y = np.mean(cbar.ax.get_ylim())
    cbar.ax.scatter(phase[miller_id], phase_y, color='k', edgecolors='w')
    spots.append(i)
    
    s = size[miller_id] * amplitude * amplitude
    ax1.scatter(x[miller_id], y[miller_id], s=s, c=phase[miller_id], cmap=phase_cmap_name, vmin=-180, vmax=180.)
    ax1.scatter(x[miller_id], y[miller_id], s=1.5*s, facecolor='None', edgecolor='w')
    
    
    dx = x.std() / 3.
    ax1.set_xlim(x[miller_id] - dx, x[miller_id] + dx)
    ax1.set_ylim(y[miller_id] - dx, y[miller_id] + dx)
    
    ax2.set_facecolor(plt.get_cmap(cmap_name)(0.))
    cell.plot_cell(ax=ax2, fmt='--w')
    cell.crop_cell(ax=ax2)
    plt.tight_layout()
    camera.snap()

plt.close() # Don't show static canvas
anim = camera.animate()
dpi = 300

HTML(anim.to_html5_video())